In [1]:
from mayavi import mlab

In [6]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import Counter
import imageio
import cv2
import sys

from IPython import display
def draw_lidar(pointcloud, color=None, fig=None, bg_color=(0, 0, 0), points_scale=0.3, points_mode="sphere",
               points_color=None, color_by_intensity=False, pointcloud_label=False):
    points_mode = "point"
    if fig is None:
        fig = mlab.figure(figure=None, bgcolor=bg_color, fgcolor=None, engine=None, size=(1500, 1000))
    if color is None:
        color = pointcloud[:, 2]
    if pointcloud_label:
        color = pointcloud[:, 4]
    if color_by_intensity:
        color = pointcloud[:, 2]
    mlab.points3d(pointcloud[:, 0], pointcloud[:, 1], pointcloud[:, 2],
                  color, color=points_color, mode=points_mode, colormap="gnuplot",
                  scale_factor=points_scale, figure=fig)  # points
    mlab.points3d(0, 0, 0, color=(1, 1, 1), mode="sphere", scale_factor=0.2)  # origin
    axes = np.array([[2.0, 0.0, 0.0, 0.0], [0.0, 2.0, 0.0, 0.0], [0.0, 0.0, 2.0, 0.0]], dtype=np.float64)
    mlab.plot3d([0, axes[0, 0]], [0, axes[0, 1]], [0, axes[0, 2]],
                color=(1, 0, 0), tube_radius=None, figure=fig)
    mlab.plot3d([0, axes[1, 0]], [0, axes[1, 1]], [0, axes[1, 2]],
                color=(0, 1, 0), tube_radius=None, figure=fig)
    mlab.plot3d([0, axes[2, 0]], [0, axes[2, 1]], [0, axes[2, 2]],
                color=(0, 0, 1), tube_radius=None, figure=fig)

    fov = np.array([[20.0, 20.0, 0.0, 0.0], [20.0, -20.0, 0.0, 0.0]], dtype=np.float64)
    mlab.plot3d([0, fov[0, 0]], [0, fov[0, 1]], [0, fov[0, 2]],
                color=(1, 1, 1), tube_radius=None, line_width=1, figure=fig)
    mlab.plot3d([0, fov[1, 0]], [0, fov[1, 1]], [0, fov[1, 2]],
                color=(1, 1, 1), tube_radius=None, line_width=1, figure=fig)

    x1, y1 = 0, -20
    x2, y2 = 40, 20
    
    mlab.plot3d([x1, x1], [y1, y2], [0, 0],
                color=(0.5, 0.5, 0.5), tube_radius=0.1, line_width=1, figure=fig)
    mlab.plot3d([x2, x2], [y1, y2], [0, 0],
                color=(0.5, 0.5, 0.5), tube_radius=0.1, line_width=1, figure=fig)
    mlab.plot3d([x1, x2], [y1, y1], [0, 0],
                color=(0.5, 0.5, 0.5), tube_radius=0.1, line_width=1, figure=fig)
    mlab.plot3d([x1, x2], [y2, y2], [0, 0],
                color=(0.5, 0.5, 0.5), tube_radius=0.1, line_width=1, figure=fig)

    mlab.view(azimuth=180, elevation=70, focalpoint=[12.0909996, -1.04700089, -2.03249991],
              distance=62.0, figure=fig)
    return fig

In [7]:
def draw_gt_boxes3d(gt_boxes3d, fig, color=(1, 1, 1)):
    """
    Draw 3D bounding boxes
    Args:
        gt_boxes3d: numpy array (3,8) for XYZs of the box corners
        fig: figure handler
        color: RGB value tuple in range (0,1), box line color
    """
    for k in range(0, 4):
        i, j = k, (k + 1) % 4
        mlab.plot3d([gt_boxes3d[0, i], gt_boxes3d[0, j]], [gt_boxes3d[1, i], gt_boxes3d[1, j]],
                    [gt_boxes3d[2, i], gt_boxes3d[2, j]], tube_radius=None, line_width=2, color=color, figure=fig)

        i, j = k + 4, (k + 1) % 4 + 4
        mlab.plot3d([gt_boxes3d[0, i], gt_boxes3d[0, j]], [gt_boxes3d[1, i], gt_boxes3d[1, j]],
                    [gt_boxes3d[2, i], gt_boxes3d[2, j]], tube_radius=None, line_width=2, color=color, figure=fig)

        i, j = k, k + 4
        mlab.plot3d([gt_boxes3d[0, i], gt_boxes3d[0, j]], [gt_boxes3d[1, i], gt_boxes3d[1, j]],
                    [gt_boxes3d[2, i], gt_boxes3d[2, j]], tube_radius=None, line_width=2, color=color, figure=fig)
    return fig 

In [9]:
mlab.init_notebook('ipy')
def display_frame_statistics():
    points  = np.fromfile(str('/home/sathish/Documents/dataset/Kitti/training/velodyne/000008.bin'),dtype=np.float32).reshape([-1, 4])
    fig=draw_lidar(points, pointcloud_label=False)

    display.display(fig)
display_frame_statistics()

Notebook initialized with ipy backend.


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\xdc\x00\x00\x03\xe8\x08\x02\x00\x00\x00Kl\xab\xb…

In [7]:
import numpy as np
import cv2
import os
import mayavi
import matplotlib.pyplot as plt
import loi as xmlParser

In [8]:
def load_tracklets_for_frames(n_frames, xml_path):
    """
    Loads dataset labels also referred to as tracklets, saving them individually for each frame.

    Parameters
    ----------
    n_frames    : Number of frames in the dataset.
    xml_path    : Path to the tracklets XML.

    Returns
    -------
    Tuple of dictionaries with integer keys corresponding to absolute frame numbers and arrays as values. First array
    contains coordinates of bounding box vertices for each object in the frame, and the second array contains objects
    types as strings.
    """
    tracklets = xmlParser.parseXML(xml_path)    
    frame_tracklets = {}
    frame_tracklets_types = {}
    for i in range(n_frames):
        frame_tracklets[i] = []
        frame_tracklets_types[i] = []

    # loop over tracklets
    for i, tracklet in enumerate(tracklets):
        # this part is inspired by kitti object development kit matlab code: computeBox3D
        h, w, l = tracklet.size
        # in velodyne coordinates around zero point and without orientation yet
        trackletBox = np.array([
            [-l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2],
            [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2],
            [0.0, 0.0, 0.0, 0.0, h, h, h, h]
        ])
        # loop over all data in tracklet
        for translation, rotation, state, occlusion, truncation, amtOcclusion, amtBorders, absoluteFrameNumber in tracklet:
            # determine if object is in the image; otherwise continue
            if truncation not in (xmlParser.TRUNC_IN_IMAGE, xmlParser.TRUNC_TRUNCATED):
                continue
            # re-create 3D bounding box in velodyne coordinate system
            yaw = rotation[2]  # other rotations are supposedly 0
            assert np.abs(rotation[:2]).sum() == 0, 'object rotations other than yaw given!'
            rotMat = np.array([
                [np.cos(yaw), -np.sin(yaw), 0.0],
                [np.sin(yaw), np.cos(yaw), 0.0],
                [0.0, 0.0, 1.0]
            ])
            cornerPosInVelo = np.dot(rotMat, trackletBox) + np.tile(translation, (8, 1)).T
            frame_tracklets[absoluteFrameNumber] = frame_tracklets[absoluteFrameNumber] + [cornerPosInVelo]
            frame_tracklets_types[absoluteFrameNumber] = frame_tracklets_types[absoluteFrameNumber] + [
                tracklet.objectType]

    return (frame_tracklets, frame_tracklets_types)

In [13]:
mlab.init_notebook('ipy')
class Object3D:
    def __init__(self, label):
        data = label.split(' ')

        self.classification = data[0]
        self.truncation = float(data[1])
        self.occlusion = int(data[2])
        self.alpha_observation_angle = float(data[3])

        self.xmin_left = float(data[4])
        self.ymin_top = float(data[5])
        self.xmax_right = float(data[6])
        self.ymax_bottom = float(data[7])

        self.bbox2d = np.array([self.xmin_left, self.ymin_top, self.xmax_right, self.ymax_bottom])

        self.height = float(data[8])
        self.width = float(data[9])
        self.length = float(data[10])

        self.obj3d_location_camera_coords = (float(data[11]), float(data[12]), float(data[13]))

        self.yaw_rotation_y = float(data[14])


class Calibration:
    def __init__(self, calib_filename, from_video=False):
        if not from_video:
            calibration = self.read_calibration_file(calib_filename)
        else:
            calibration = self.read_calibration_from_video(calib_filename)
        self.P = calibration["P2"]
        self.P = np.reshape(self.P, [3, 4])
        self.V2C = calibration["Tr_velo_to_cam"]
        self.V2C = np.reshape(self.V2C, [3, 4])
        self.C2V = inverse_rigid_transform(self.V2C)
        self.R0 = calibration["R0_rect"]
        self.R0 = np.reshape(self.R0, [3, 3])

        self.c_u = self.P[0, 2]
        self.c_v = self.P[1, 2]
        self.f_u = self.P[0, 0]
        self.f_v = self.P[1, 1]
        self.b_x = self.P[0, 3] / (-self.f_u)
        self.b_y = self.P[1, 3] / (-self.f_v)

    def read_calibration_file(self, filename):
        data = {}
        with open(filename, "r") as file:
            for line in file.readlines():
                line = line.rstrip()
                if len(line) == 0:
                    continue
                key, value = line.split(":", 1)
                try:
                    data[key] = np.array([float(x) for x in value.split()])
                except ValueError:
                    pass
        return data

    def read_calibration_from_video(self, calib_dir):
        data = {}
        cam2cam = self.read_calibration_file(os.path.join(calib_dir, "calib_cam_to_cam.txt"))
        velo2cam = self.read_calibration_file(os.path.join(calib_dir, "calib_velo_to_cam.txt"))

        Tr_velo_to_cam = np.zeros((3, 4))
        Tr_velo_to_cam[0:3, 0:3] = np.reshape(velo2cam["R"], [3, 3])
        Tr_velo_to_cam[:, 3] = velo2cam["T"]

        data["Tr_velo_to_cam"] = np.reshape(Tr_velo_to_cam, [12])
        data["R0_rect"] = cam2cam["R_rect_00"]
        data["P2"] = cam2cam["P_rect_02"]

        return data

    def project_lidar_to_image(self, points3d_lidar):
        points3d_rect = self.project_lidar_to_rect(points3d_lidar)
        return self.project_rect_to_image(points3d_rect)

    def project_lidar_to_rect(self, points3d_lidar):
        points3d_ref = self.project_lidar_to_ref(points3d_lidar)
        return self.project_ref_to_rect(points3d_ref)

    def project_lidar_to_ref(self, points3d_lidar):
        points3d_lidar = self.cart2hom(points3d_lidar)
        return np.dot(points3d_lidar, np.transpose(self.V2C))

    def project_ref_to_rect(self, points3d_ref):
        return np.transpose(np.dot(self.R0, np.transpose(points3d_ref)))

    def cart2hom(self, points3d):
        """In: nx3 - Points in Cartesian
           Out: nx4 - Points in Homogeneous
        """
        num_points = points3d.shape[0]
        points3d_hom = np.hstack((points3d, np.ones((num_points, 1))))
        return points3d_hom

    def project_rect_to_image(self, points3d_rect):
        points3d_rect = self.cart2hom(points3d_rect)
        points2d = np.dot(points3d_rect, np.transpose(self.P))
        points2d[:, 0] /= points2d[:, 2]
        points2d[:, 1] /= points2d[:, 2]
        return points2d[:, 0:2]

    def project_rect_to_lidar(self, points3d_rect):
        points3d_ref = self.project_rect_to_ref(points3d_rect)
        return self.project_ref_to_lidar(points3d_ref)

    def project_rect_to_ref(self, points3d_rect):
        return np.transpose(np.dot(np.linalg.inv(self.R0), np.transpose(points3d_rect)))

    def project_ref_to_lidar(self, points3d_ref):
        points3d_ref = self.cart2hom(points3d_ref)
        return np.dot(points3d_ref, np.transpose(self.C2V))


class KITTIVideo:
    def __init__(self, img_dir, lidar_dir, calib_dir):
        self.calib = Calibration(calib_dir, from_video=True)
        self.img_dir = img_dir
        self.lidar_dir = lidar_dir

        self.img_filenames = sorted([os.path.join(img_dir, filename) for filename in os.listdir(img_dir)])
        self.lidar_filenames = sorted([os.path.join(lidar_dir, filename) for filename in os.listdir(lidar_dir)])

        self.num_samples = len(self.img_filenames)

    def __len__(self):
        return self.num_samples

    def get_image(self, idx):
        return load_image(self.img_filenames[idx])

    def get_lidar(self, idx):
        return get_velodyne_scan_points(self.lidar_filenames[idx])

    def get_calibration(self):
        return self.calib


def inverse_rigid_transform(tr):
    inv_tr = np.zeros_like(tr)
    inv_tr[0:3, 0:3] = np.transpose(tr[0:3, 0:3])
    inv_tr[0:3, 3] = np.dot(-np.transpose(tr[0:3, 0:3]), tr[0:3, 3])
    return inv_tr


def load_image(image_filename):
    return cv2.imread(image_filename)


def read_label(label_file_name):
    objects = []
    with open(label_file_name, 'r') as lf:
        for line in lf.readlines():
            objects.append(Object3D(line.rstrip()))
    return objects


def rotate_y(p):
    cp = np.cos(p)
    sp = np.sin(p)

    return np.array([[cp, 0, sp], [0, 1, 0], [-sp, 0, cp]])


def project_3d_to_image2d(points3d, P):
    n = points3d.shape[0]
    points3d_extended = np.hstack((points3d, np.ones((n, 1))))
    points2d = np.dot(points3d_extended, np.transpose(P))
    points2d[:, 0] /= points2d[:, 2]
    points2d[:, 1] /= points2d[:, 2]
    return points2d[:, 0:2]


def draw_projected_bbox3d(image, vertices, color, thickness):
    vertices = vertices.astype(np.int32)
    for idx in range(0, 4):
        i, j = idx, (idx + 1) % 4
        cv2.line(image, (vertices[i, 0], vertices[i, 1]),
                 (vertices[j, 0], vertices[j, 1]),
                 color, thickness)
        i, j = idx + 4, (idx + 1) % 4 + 4
        cv2.line(image, (vertices[i, 0], vertices[i, 1]),
                 (vertices[j, 0], vertices[j, 1]),
                 color, thickness)
        i, j = idx, idx + 4
        cv2.line(image, (vertices[i, 0], vertices[i, 1]),
                 (vertices[j, 0], vertices[j, 1]),
                 color, thickness)
    return image


def compute_bbox3d(obj, P):
    R = rotate_y(obj.yaw_rotation_y)

    l = obj.length
    w = obj.width
    h = obj.height

    # 3D BBox Corners
    x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2]
    y_corners = [0, 0, 0, 0, -h, -h, -h, -h]
    z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2]

    # rotate and translate 3D BBox
    corners3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))

    corners3d[0, :] = corners3d[0, :] + obj.obj3d_location_camera_coords[0]
    corners3d[1, :] = corners3d[1, :] + obj.obj3d_location_camera_coords[1]
    corners3d[2, :] = corners3d[2, :] + obj.obj3d_location_camera_coords[2]

    if np.any(corners3d[2, :] < 0.1): # Draw 3D BBoxes for objects which are infront of Camera
        corners2d = None
        return corners2d, np.transpose(corners3d)

    # Project 3D BBox onto Image plane
    corners2d = project_3d_to_image2d(np.transpose(corners3d), P)
    return corners2d, np.transpose(corners3d)


def get_velodyne_scan_points(lidar_filename, n_vec=4):
    lidar_scanpoints = np.fromfile(lidar_filename, np.float32)
    lidar_scanpoints = lidar_scanpoints.reshape((-1, n_vec))
    return lidar_scanpoints
def draw_groundtruth_3dbboxes(groundtruth_3dbboxes, fig, color=(1, 1, 1), line_width=1, draw_text=True,
                              text_scale=(1, 1, 1), color_list=None, label=""):
    num_bboxes = len(groundtruth_3dbboxes)
    for idx in range(num_bboxes):
        bbox = groundtruth_3dbboxes[idx]
        if color_list is not None:
            color = color_list[idx]
        if draw_text:
            mlab.text3d(bbox[4, 0], bbox[4, 1], bbox[4, 2], label, scale=text_scale, color=color, figure=fig)
        for k in range(0, 4):
            i, j = k, (k + 1) % 4
            mlab.plot3d([bbox[i, 0], bbox[j, 0]],
                        [bbox[i, 1], bbox[j, 1]],
                        [bbox[i, 2], bbox[j, 2]],
                        color=color,
                        tube_radius=None,
                        line_width=line_width,
                        figure=fig)
            i, j = k + 4, (k + 1) % 4 + 4
            mlab.plot3d([bbox[i, 0], bbox[j, 0]],
                        [bbox[i, 1], bbox[j, 1]],
                        [bbox[i, 2], bbox[j, 2]],
                        color=color,
                        tube_radius=None,
                        line_width=line_width,
                        figure=fig)
            i, j = k, k + 4
            mlab.plot3d([bbox[i, 0], bbox[j, 0]],
                        [bbox[i, 1], bbox[j, 1]],
                        [bbox[i, 2], bbox[j, 2]],
                        color=color,
                        tube_radius=None,
                        line_width=line_width,
                        figure=fig)
    return fig


def draw_lidar_simple(pointcloud, color=None):
    figure = mlab.figure(figure=None, bgcolor=(0, 0, 0), fgcolor=None, engine=None, size=(1500, 1000))
    if color is None:
        color = pointcloud[:, 2]
    mlab.points3d(pointcloud[:, 0], pointcloud[:, 1], pointcloud[:, 2],
                  color, color=None, mode="point", colormap="gnuplot", scale_factor=1, figure=figure)  # points
    mlab.points3d(0, 0, 0, color=(1, 1, 1), mode="sphere", scale_factor=0.2)  # origin
    axes = np.array([[2.0, 0.0, 0.0, 0.0], [0.0, 2.0, 0.0, 0.0], [0.0, 0.0, 2.0, 0.0]], dtype=np.float64)
    mlab.plot3d([0, axes[0, 0]], [0, axes[0, 1]], [0, axes[0, 2]],
                color=(1, 0, 0), tube_radius=None, figure=figure)
    mlab.plot3d([0, axes[1, 0]], [0, axes[1, 1]], [0, axes[1, 2]],
                color=(0, 1, 0), tube_radius=None, figure=figure)
    mlab.plot3d([0, axes[2, 0]], [0, axes[2, 1]], [0, axes[2, 2]],
                color=(0, 0, 1), tube_radius=None, figure=figure)
    mlab.view(azimuth=180, elevation=70, focalpoint=[12.0909996, -1.04700089, -2.03249991],
              distance=62.0, figure=figure)
    return figure


def draw_lidar(pointcloud, color=None, fig=None, bg_color=(0, 0, 0), points_scale=0.3, points_mode="sphere",
               points_color=None, color_by_intensity=False, pointcloud_label=False):
    points_mode = "point"
    if fig is None:
        fig = mlab.figure(figure=None, bgcolor=bg_color, fgcolor=None, engine=None, size=(1500, 1000))
    if color is None:
        color = pointcloud[:, 2]
    if pointcloud_label:
        color = pointcloud[:, 4]
    if color_by_intensity:
        color = pointcloud[:, 2]
    mlab.points3d(pointcloud[:, 0], pointcloud[:, 1], pointcloud[:, 2],
                  color, color=points_color, mode=points_mode, colormap="gnuplot",
                  scale_factor=points_scale, figure=fig)  # points
    mlab.points3d(0, 0, 0, color=(1, 1, 1), mode="sphere", scale_factor=0.2)  # origin
    axes = np.array([[2.0, 0.0, 0.0, 0.0], [0.0, 2.0, 0.0, 0.0], [0.0, 0.0, 2.0, 0.0]], dtype=np.float64)
    mlab.plot3d([0, axes[0, 0]], [0, axes[0, 1]], [0, axes[0, 2]],
                color=(1, 0, 0), tube_radius=None, figure=fig)
    mlab.plot3d([0, axes[1, 0]], [0, axes[1, 1]], [0, axes[1, 2]],
                color=(0, 1, 0), tube_radius=None, figure=fig)
    mlab.plot3d([0, axes[2, 0]], [0, axes[2, 1]], [0, axes[2, 2]],
                color=(0, 0, 1), tube_radius=None, figure=fig)

    fov = np.array([[20.0, 20.0, 0.0, 0.0], [20.0, -20.0, 0.0, 0.0]], dtype=np.float64)
    mlab.plot3d([0, fov[0, 0]], [0, fov[0, 1]], [0, fov[0, 2]],
                color=(1, 1, 1), tube_radius=None, line_width=1, figure=fig)
    mlab.plot3d([0, fov[1, 0]], [0, fov[1, 1]], [0, fov[1, 2]],
                color=(1, 1, 1), tube_radius=None, line_width=1, figure=fig)

    x1, y1 = 0, -20
    x2, y2 = 40, 20
    
    mlab.plot3d([x1, x1], [y1, y2], [0, 0],
                color=(0.5, 0.5, 0.5), tube_radius=0.1, line_width=1, figure=fig)
    mlab.plot3d([x2, x2], [y1, y2], [0, 0],
                color=(0.5, 0.5, 0.5), tube_radius=0.1, line_width=1, figure=fig)
    mlab.plot3d([x1, x2], [y1, y1], [0, 0],
                color=(0.5, 0.5, 0.5), tube_radius=0.1, line_width=1, figure=fig)
    mlab.plot3d([x1, x2], [y2, y2], [0, 0],
                color=(0.5, 0.5, 0.5), tube_radius=0.1, line_width=1, figure=fig)

    mlab.view(azimuth=180, elevation=70, focalpoint=[12.0909996, -1.04700089, -2.03249991],
              distance=62.0, figure=fig)
    return fig



class KITTIObject:
    def __init__(self, root_dir, dataset_type, ):
        self.root_dir = root_dir
        self.dataset_type = dataset_type

        self.dataset_dir = os.path.join(self.root_dir, self.dataset_type)

        self.num_samples = len(os.listdir(self.dataset_dir))

        self.image_dir = os.path.join(self.dataset_dir, "image_2")
        self.label_dir = os.path.join(self.dataset_dir, "label_2")
        self.calib_dir = os.path.join(self.dataset_dir, "calib")
        self.lidar_dir = os.path.join(self.dataset_dir, "velodyne")

    def get_label_objects(self, idx):
        if idx > self.num_samples:
            return None
        label_filename = os.path.join(self.label_dir, "%06d.txt" % idx)
        return read_label(label_filename)

    def get_lidar_data(self, idx, n_vec=4):
        lidar_filename = os.path.join(self.lidar_dir, "%06d.bin" % idx)
        return get_velodyne_scan_points(lidar_filename, n_vec)

    def get_calibration(self, idx):
        calib_filename = os.path.join(self.calib_dir, "%06d.txt" % idx)
        return Calibration(calib_filename)

    def get_image(self, idx):
        image_filename = os.path.join(self.image_dir, "%06d.png" % idx)
        return load_image(image_filename)


def get_lidar_index_in_image_fov(pcd, calib, xmin, ymin, xmax, ymax, clip_distance=2.0):
    points2d = calib.project_lidar_to_image(pcd)
    fov_idxs = ((points2d[:, 0] < xmax) & (points2d[:, 0] >= xmin) &
                (points2d[:, 1] < ymax) & (points2d[:, 1] >= ymin))
    fov_idxs = fov_idxs & (pcd[:, 0] > clip_distance)
    return fov_idxs


def show_lidar_data(point_cloud_data, objects, calibration, figure,
                    img_fov=False, img_width=None, img_height=None, cam_img=None, pc_label=False, save=False):
    # print(("All point num: ",
    point_cloud_data.shape[0]
    if img_fov:
        pcd_index = get_lidar_index_in_image_fov(point_cloud_data[:, :3], calibration, 0, 0, img_width, img_height)
        point_cloud_data = point_cloud_data[pcd_index, :]
        # print(("FOV point num: ", point_cloud_data.shape))
    draw_lidar(point_cloud_data, fig=figure, pointcloud_label=pc_label)

    color = (0, 1, 0)

    if objects is not None:
        for obj in objects:
            if obj.classification == "DontCare":
                continue
            _, bbox3d = compute_bbox3d(obj, calibration.P)
            bbox3d_velodyne = calibration.project_rect_to_lidar(bbox3d)
            draw_groundtruth_3dbboxes([bbox3d_velodyne], fig=figure, color=color, label=obj.classification)
        mlab.show(1)


def get_lidar_data_in_image_fov(pcd, calib, xmin, ymin, xmax, ymax, clip_distance=2.0):
    points2d = calib.project_lidar_to_image(pcd)
    fov_idxs = ((points2d[:, 0] < xmax) & (points2d[:, 0] >= xmin) &
                (points2d[:, 1] < ymax) & (points2d[:, 1] >= ymin))
    fov_idxs = fov_idxs & (pcd[:, 0] > clip_distance)
    imgfov_pcd = pcd[fov_idxs, :]

    return imgfov_pcd, points2d, fov_idxs


def show_lidar_data_on_image(point_cloud_data, img, calib, img_width, img_height):
    img = np.copy(img)
    imgfov_pcd, points2d, fov_indices = \
        get_lidar_data_in_image_fov(point_cloud_data, calib, 0, 0, img_width, img_height)
    imgfov_points2d = points2d[fov_indices, :]
    imgfov_pc_rect = calib.project_lidar_to_rect(imgfov_pcd)

    cmap = plt.cm.get_cmap("hsv", 256)
    cmap = np.array([cmap(i) for i in range(256)])[:, :3] * 255

    for i in range(imgfov_points2d.shape[0]):
        depth = imgfov_pc_rect[i, 2]
        color = cmap[int(640.0 / depth), :]
        cv2.circle(img, (int(np.round(imgfov_points2d[i, 0])), int(np.round(imgfov_points2d[i, 1]))),
                   2, color=tuple(color), thickness=-1)
    return img


def show_bboxes_on_image(image, objects, calib):
    img_2d = image.copy()
    img_3d = image.copy()

    if objects is not None:
        for obj in objects:
            if obj.classification == "DontCare":
                continue
            cv2.rectangle(img_2d, (int(obj.xmin_left), int(obj.ymin_top)),
                          (int(obj.xmax_right), int(obj.ymax_bottom)),
                          (0, 255, 0), 3)
            bbox3d_points2d, _ = compute_bbox3d(obj, calib.P)
            img_3d = draw_projected_bbox3d(img_3d, bbox3d_points2d, (0, 255, 0), 3)
    return img_2d, img_3d


def visualize_video(img_dir, lidar_dir, calib_dir, output_dir):
    dataset = KITTIVideo(img_dir, lidar_dir, calib_dir)

    fig = None

    for i in range(len(dataset)):
        img = dataset.get_image(i)
        pcd = dataset.get_lidar(i)
        # cv2.imshow("Video", img)
        fig = draw_lidar(pcd, pointcloud_label=False)

        mlab.savefig(os.path.join(output_dir, "P%s.png" % i))

    return fig  # return last figure

Notebook initialized with ipy backend.


In [10]:

root_path = r"/home/sathish/Downloads/2011_09_26/2011_09_26/2011_09_26"

lidar_path = r"/home/sathish/Downloads/2011_09_26/2011_09_26/2011_09_26/2011_09_26_drive_0001_sync/velodyne_points/data"

In [11]:
from IPython import display
dataset = KITTIObject(r'/home/sathish/Documents/dataset/Kitti', 'training')
data_idx = 1
idx=1
objects = dataset.get_label_objects(idx)
pcd = dataset.get_lidar_data(idx)
calib = dataset.get_calibration(idx)
img = dataset.get_image(idx)
img_height, img_width, _ = img.shape
fig_3d = mlab.figure(bgcolor=(0, 0, 0), size=(800, 500))
show_lidar_data(pcd, objects, calib, fig_3d, False, img_width, img_height,save=True)
display.display(fig_3d)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03 \x00\x00\x01\xf4\x08\x02\x00\x00\x00J"\xde/\x00\…

In [ ]:
image_dir = r"/home/sathish/Downloads/2011_09_26/2011_09_26/2011_09_26/2011_09_26_drive_0001_sync/image_02/data"
lidar_dir = r"/home/sathish/Downloads/2011_09_26/2011_09_26/2011_09_26/2011_09_26_drive_0001_sync/velodyne_points/data"
calib_dir = r"/home/sathish/Downloads/2011_09_26/2011_09_26/2011_09_26"
output_path = r"/home/sathish/Output"
fig_3d = visualize_video(image_dir, lidar_dir, calib_dir, output_path)
display.display(fig_3d)

Parsing tracklet file /home/sathish/Downloads/2011_09_26/2011_09_26/2011_09_26_drive_0048_sync/tracklet_labels.xml
File contains 8 tracklets
Loaded 8 tracklets.


In [44]:
tracklet_rects[21]

[array([[ 3.61872747,  3.57328205,  7.34074598,  7.3861914 ,  3.61872747,
          3.57328205,  7.34074598,  7.3861914 ],
        [-1.78385112, -3.4259711 , -3.5302351 , -1.88811512, -1.78385112,
         -3.4259711 , -3.5302351 , -1.88811512],
        [-1.68103695, -1.68103695, -1.68103695, -1.68103695, -0.26733065,
         -0.26733065, -0.26733065, -0.26733065]]),
 array([[32.64796338, 32.66172576, 28.68707403, 28.67331166, 32.64796338,
         32.66172576, 28.68707403, 28.67331166],
        [ 3.64524225,  5.40046889,  5.43163333,  3.67640668,  3.64524225,
          5.40046889,  5.43163333,  3.67640668],
        [-1.65444398, -1.65444398, -1.65444398, -1.65444398, -0.22131778,
         -0.22131778, -0.22131778, -0.22131778]]),
 array([[12.00426498, 12.04482511,  8.10180148,  8.06124135, 12.00426498,
         12.04482511,  8.10180148,  8.06124135],
        [ 4.04690329,  5.65158057,  5.7512452 ,  4.14656792,  4.04690329,
          5.65158057,  5.7512452 ,  4.14656792],
        [-1.